In [28]:
for var in list(globals()):
    if var != "__name__":
        del globals()[var]
        
import pandas as pd
import numpy as np
import json
import os
from tqdm import tqdm
from global_variables import *
import gzip
import gc

In [29]:
visit_summary_file = 'C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/place_month_visit_summary.csv.gz'
ft_data_path = 'C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/filtered_ft_data.csv.gz'

visit_summary = pd.read_csv(visit_summary_file,sep="|", compression='gzip')
visit_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1573552 entries, 0 to 1573551
Data columns (total 8 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   PLACEKEY            1573552 non-null  object 
 1   DATE_RANGE_START    1573552 non-null  object 
 2   RAW_VISIT_COUNTS    1573552 non-null  float64
 3   RAW_VISITOR_COUNTS  1573552 non-null  float64
 4   DISTANCE_FROM_HOME  1573552 non-null  float64
 5   MEDIAN_DWELL        1569197 non-null  float64
 6   android             1573552 non-null  int64  
 7   ios                 1573552 non-null  int64  
dtypes: float64(4), int64(2), object(2)
memory usage: 96.0+ MB


In [30]:
visits_file = 'C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/cbgs_place_month_visits_count.csv.gz'

visits = pd.read_csv(visits_file,sep="|", compression='gzip',low_memory=False)
visits['censustract'] = visits['CBGS'].str[:-1]
visits = visits.drop('CBGS', axis=1)
visits = visits.groupby(['censustract','PLACEKEY','DATE_RANGE_START']).sum().reset_index() 
visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31990377 entries, 0 to 31990376
Data columns (total 4 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   censustract       object
 1   PLACEKEY          object
 2   DATE_RANGE_START  object
 3   no_visits         int64 
dtypes: int64(1), object(3)
memory usage: 976.3+ MB


In [35]:
acs_block_data_file = "C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/acs_tract_data_2.csv.gz"

asc_block_data = pd.read_csv(acs_block_data_file, compression='gzip')

asc_block_data['state'] = asc_block_data['state'].astype(str).str.zfill(2)
asc_block_data['county'] = asc_block_data['county'].astype(str).str.zfill(3)
asc_block_data['tract'] = asc_block_data['tract'].astype(str).str.zfill(6)
#asc_block_data['block group'] = asc_block_data['block group'].astype(str)

asc_block_data['median_age'] = np.where(asc_block_data['median_age'] < 0, np.nan, asc_block_data['median_age'])
asc_block_data['median_income_last_12_months'] = np.where(asc_block_data['median_income_last_12_months'] < 0, np.nan, asc_block_data['median_income_last_12_months'])
asc_block_data['aggregate_family_income'] = np.where(asc_block_data['aggregate_family_income'] < 0, np.nan, asc_block_data['aggregate_family_income'])

asc_block_data['censustract'] = asc_block_data['state'] + asc_block_data['county'] + asc_block_data['tract']#+asc_block_data['block group']


asc_block_data['mean_family_income'] = asc_block_data['aggregate_family_income']/asc_block_data['no_of_households']

asc_block_data['white_fraction'] = asc_block_data['population_race_white']/asc_block_data['population_race_base']

asc_block_data['english_speaking_frac'] = asc_block_data['households_language_english']/asc_block_data['households_language_base']

asc_block_data['spanish_speaking_frac'] = asc_block_data['households_language_spanish']/asc_block_data['households_language_base']

asc_block_data['internet_frac'] =asc_block_data['households_internet_yes']/asc_block_data['households_internet_base']

asc_block_data['computer_frac'] =asc_block_data['households_computer_yes']/asc_block_data['households_computer_base']

asc_block_data['under_19_frac'] =asc_block_data['population_by_age_base_under_19']/asc_block_data['population_by_age_base']

asc_block_data['over_65_frac'] = asc_block_data['population_by_age_base_65_older']/asc_block_data['population_by_age_base']

asc_block_data['below_poverty_frac'] = asc_block_data['poverty_status_below']/asc_block_data['poverty_status_base']

asc_block_data['food_stamp_frac'] = asc_block_data['received_food_stamps_snap_yes']/asc_block_data['received_food_stamps_snap_base']

asc_block_data['owner_occ_frac'] = asc_block_data['housing_units_occupancy_own']/asc_block_data['housing_units_occupancy_base']

asc_block_data['home_with_mtg_frac'] = asc_block_data['housing_units_with_a_mortgage']/asc_block_data['housing_units_mortgage_base']

asc_block_data['college_or_higher_frac'] = asc_block_data['education_college_or_higher']/asc_block_data['education_base']

asc_block_data['less_than_high_school_frac'] = asc_block_data['education_less_than_high_school']/asc_block_data['education_base']

selected_columns = [
    'censustract',
    'median_age',
    'median_income_last_12_months',
    'mean_family_income',
    'white_fraction',
    'travel_time_to_work',
    'spanish_speaking_frac',
    'english_speaking_frac',
    'internet_frac',
    'computer_frac',
    'under_19_frac',
    'over_65_frac',
    'below_poverty_frac',
    'food_stamp_frac',
    'owner_occ_frac',
    'college_or_higher_frac',
    'less_than_high_school_frac',
]

acs = asc_block_data[selected_columns]
acs.reset_index(inplace=True)

acs = acs.copy()
acs.info()


In [40]:
new_acs = acs.copy()
new_acs['censustract'] = new_acs['censustract'].str[:-1] + '0'
new_acs = new_acs.groupby('censustract').mean().reset_index() 
mask = ~new_acs['censustract'].isin(acs['censustract'])

combined_acs = pd.concat([acs, new_acs[mask]], ignore_index=True)
combined_acs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100154 entries, 0 to 100153
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   index                         100154 non-null  float64
 1   censustract                   100154 non-null  object 
 2   median_age                    99263 non-null   float64
 3   median_income_last_12_months  99089 non-null   float64
 4   mean_family_income            98767 non-null   float64
 5   white_fraction                99309 non-null   float64
 6   travel_time_to_work           100154 non-null  float64
 7   spanish_speaking_frac         99050 non-null   float64
 8   english_speaking_frac         99050 non-null   float64
 9   internet_frac                 99050 non-null   float64
 10  computer_frac                 99050 non-null   float64
 11  under_19_frac                 99158 non-null   float64
 12  over_65_frac                  99158 non-null

In [42]:
combined_acs = combined_acs.drop('index', axis=1)
combined_acs = combined_acs.replace([np.inf, -np.inf], np.nan)

In [43]:
visits_merged = pd.merge(visits, combined_acs, on='censustract', how='inner')
visits_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27253061 entries, 0 to 27253060
Data columns (total 20 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   censustract                   object 
 1   PLACEKEY                      object 
 2   DATE_RANGE_START              object 
 3   no_visits                     int64  
 4   median_age                    float64
 5   median_income_last_12_months  float64
 6   mean_family_income            float64
 7   white_fraction                float64
 8   travel_time_to_work           float64
 9   spanish_speaking_frac         float64
 10  english_speaking_frac         float64
 11  internet_frac                 float64
 12  computer_frac                 float64
 13  under_19_frac                 float64
 14  over_65_frac                  float64
 15  below_poverty_frac            float64
 16  food_stamp_frac               float64
 17  owner_occ_frac                float64
 18  college_or_higher_fr

In [45]:
with gzip.open('C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/visits_acs_merged.gz', 'wt', encoding='utf-8') as gzipped_file:
    visits_merged.to_csv(gzipped_file, index=False)

In [44]:
random_sample = visits_merged.sample(n=100, random_state=42)

In [ ]:
# temp = visits_merged[visits_merged['PLACEKEY']=='222-227@8dg-gtw-nkf']
# temp.info()

In [22]:
# columns_to_average = [col for col in visits_merged.columns if col not in ['CBGS_tract','PLACEKEY','DATE_RANGE_START','no_visits']]
# 
# def weighted_average(df):
#     # total_visits = df['no_visits'].sum()
#     weighted_averages = {}
#     
#     for col in columns_to_average:
#         total_weight = df.loc[~df[col].isna(), 'no_visits'].sum()
#         weighted_avg =  np.nansum(df[col] * df['no_visits'])/total_weight if total_weight != 0 else np.nan
#         weighted_averages[f'wa_{col}'] = weighted_avg
#     
#     return pd.Series(weighted_averages) 

In [ ]:
# weighted_avg_df = visits_merged.groupby(['PLACEKEY', 'DATE_RANGE_START']).apply(weighted_average).reset_index()
# print(weighted_avg_df.head())

In [ ]:
# weighted_avg_df.info()

In [25]:
# with gzip.open('C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/wa_acs_place_month_2.gz', 'wt', encoding='utf-8') as gzipped_file:
#     weighted_avg_df.to_csv(gzipped_file, index=False)

following section cleans ft data into visits and visit_summary

In [1]:
for var in list(globals()):
    if var != "__name__":
        del globals()[var]
        
import pandas as pd
import json
import os
from tqdm import tqdm
import gzip
from global_variables import *

In [2]:
# ft_poi_sod_clean.ipynb creates this file
ft_data_path = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/filtered_ft_data.csv.gz'
ft = pd.read_csv(ft_data_path,compression='gzip')

In [3]:
visits_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/cbgs_place_month_visits_count.csv'
visit_summary_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/place_month_visit_summary.csv'
header_option_visits_file = True  
header_option_visit_summary_file = True 

with tqdm(total=len(ft)) as pbar:
    
    for index in range(0, len(ft)):
        
        temp = ft.loc[index]
        
        data = json.loads(temp['VISITOR_HOME_CBGS'])
        df1 = pd.DataFrame(data.items(), columns=['CBGS', 'no_visits'])
        df1['PLACEKEY'] = temp['PLACEKEY']
        df1['DATE_RANGE_START'] = temp['DATE_RANGE_START']
        
        df2 = temp[['PLACEKEY','DATE_RANGE_START','RAW_VISIT_COUNTS','RAW_VISITOR_COUNTS','DISTANCE_FROM_HOME','MEDIAN_DWELL']]
        df2['android'] = 0
        df2['ios'] = 0
        
        try:
            dt = json.loads(temp['DEVICE_TYPE'])
            df2['android'] = dt['android']
        except:
            pass
        
        try:
            dt = json.loads(temp['DEVICE_TYPE'])
            df2['ios'] = dt['ios']
        except:
            pass
        df2_transposed = df2.to_frame().reset_index().T
        df2_transposed.columns = df2_transposed.iloc[0]
        df2_transposed = df2_transposed[1:]
        
        if os.path.isfile(visits_file):
            header_option_visits_file = False  
        if os.path.isfile(visit_summary_file):
            header_option_visit_summary_file = False  
    
        df1.to_csv(visits_file, index=False, mode='a', header=header_option_visits_file,sep="|")
        df2_transposed.to_csv(visit_summary_file, index=False, mode='a', header=header_option_visit_summary_file,sep="|")
        
        pbar.update(1)

100%|██████████| 1573552/1573552 [2:45:37<00:00, 158.35it/s]  


In [6]:
visits = pd.read_csv(visits_file)
visits_summary = pd.read_csv(visit_summary_file)

In [ ]:
with gzip.open(visits_file+'.gz', 'wt', encoding='utf-8') as gzipped_file:
    visits.to_csv(gzipped_file, index=False)

with gzip.open(visit_summary_file+'.gz', 'wt', encoding='utf-8') as gzipped_file:
    visits_summary.to_csv(gzipped_file, index=False)